In [47]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime



# Define the URL and tickers

market_insiderUrl = 'https://markets.businessinsider.com/stocks/'
tickers = ['AMZN','TSLA','META','MSFT','GOOG','NVDA','AAPL']

news_tables = {}
for ticker in tickers:
    news_table=[]
    for i in range(1,6):
        url = market_insiderUrl + ticker + '-stock/news?p='+str(i)



        # Use BeautifulSoup to parse the loaded page
        print(url)
        response = requests.get(url)
        response.raise_for_status()  # This will raise an exception for HTTP errors

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        news_table.extend(soup.findAll('a', class_='news-link'))
    news_tables[ticker] = news_table

print(len(news_tables['MSFT']))




https://markets.businessinsider.com/stocks/AMZN-stock/news?p=1
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=2
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=3
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=4
https://markets.businessinsider.com/stocks/AMZN-stock/news?p=5
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=1
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=2
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=3
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=4
https://markets.businessinsider.com/stocks/TSLA-stock/news?p=5
https://markets.businessinsider.com/stocks/META-stock/news?p=1
https://markets.businessinsider.com/stocks/META-stock/news?p=2
https://markets.businessinsider.com/stocks/META-stock/news?p=3
https://markets.businessinsider.com/stocks/META-stock/news?p=4
https://markets.businessinsider.com/stocks/META-stock/news?p=5
https://markets.businessinsider.com/stocks/MSFT-stock/n

In [48]:
print(len(news_tables['AMZN']))

250


In [52]:
parsed_data = []

p_data =[];



for ticker, news_table in news_tables.items():
    print(ticker)
    for elem in news_table:

        try:
            articleInfo = ''
            if elem:
               
                link = elem.get('href')
                newUrl = 'https://markets.businessinsider.com' + link 

                # Use Selenium to navigate to the new URL
                response = requests.get(newUrl)
                response.raise_for_status()  # This will raise an exception for HTTP errors

                # Parse the HTML using BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')
                h1tag = soup.find('h1',class_='article-title')
                title = h1tag.text
                
                date = soup.find('span',class_='news-post-quotetime warmGrey')
                print(date)
                date = date.text
                date = date.split(',')
                date = date[0] + ','+ date[1];
                
                date_format = "%b. %d, %Y"

                # Convert the string to a datetime object
                date_object = datetime.strptime(date, date_format)
                iso_format_date = date_object.isoformat()


                # Format the date in the standard ISO 8601 format

                content = soup.find('div', class_='col-xs-12 news-content no-padding')
                irrelevant_phrases = ["InvestorPlace - Stock Market News","Shutterstock","Source:","See also", "Read next", "Read also", "TipRanks has tracked 36,000", "Now Read", "Read now"]

                if content:
                # Extract all paragraphs, ignoring nested formatting tags by getting the .text
                    p_tags = content.find_all('p')
                    restIrelevant = False
                    pList=[]
                    for p in p_tags:
                        relevant = True
                        
                        if(len(p.text.strip())==0):
                            continue;
                        

                        if(len(p.text.strip().split())<7):
                            relevant=False
                        if(p.text.strip()=="InvestorPlace - Stock Market News, Stock Advice & Trading Tips"):
                            relevant=False
                        if(restIrelevant):
                            relevant = False
                            
                        if 'InvestorPlace.com'.lower() in p.text.strip().lower():
                            relevant = False
                            restIrelevant=True
            
                        for phrase in irrelevant_phrases:
                            if phrase.lower() in p.text.strip().lower(): 
                                relevant = False
                                break
                        pList.append(p.text.strip())
                          
                        p_data.append([ticker,link,p.text.strip(),relevant])
                    
                    articleInfo = ' '.join(p.text.strip() for p in p_tags if len(p.text.strip()) != 0)

                parsed_data.append([ticker, title,iso_format_date, link, articleInfo,pList])
                
        except Exception as e:
            print(f"An error occurred while fetching or parsing the article: {e}")
            continue  # Skip to the next loop iteration if an error occurs

df = pd.DataFrame(parsed_data, columns=['ticker', 'title','date', 'link', 'articleInfo','paragraphList'])
p_df = pd.DataFrame(p_data, columns=['ticker','link','paragraph','relevant'])

print(df.head())


AMZN
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 04:45 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 03:03 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 11:45 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 08:56 AM</span>
An error occurred while fetching or parsing the article: HTTPSConnectionPool(host='markets.businessinsider.comhttps', port=443): Max retries exceeded with url: //seekingalpha.com/news/4089996-fund-manager-bullishness-nears-close-your-eyes-and-sell-level?utm_source=businessinsider&utm_medium=referral&feed_item_type=news (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1294038d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 06:25 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 04:52 AM</span>
<span class="news-post-quotetime warmGrey">

<span class="news-post-quotetime warmGrey">Apr. 5, 2024, 11:21 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 5, 2024, 04:37 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 5, 2024, 04:21 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 5, 2024, 04:12 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 5, 2024, 03:16 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 03:35 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 03:01 PM</span>
An error occurred while fetching or parsing the article: HTTPSConnectionPool(host='markets.businessinsider.comhttps', port=443): Max retries exceeded with url: //seekingalpha.com/news/4087410-feds-kashkari-bitcoin-has-had-no-legitimate-use-case-in-more-than-a-decade?utm_source=businessinsider&utm_medium=referral&feed_item_type=news (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x128275850>: Failed to establish a new connection: [Errno 8] nodename nor ser

<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 02:30 PM</span>
An error occurred while fetching or parsing the article: HTTPSConnectionPool(host='markets.businessinsider.comhttps', port=443): Max retries exceeded with url: //seekingalpha.com/news/4083927-amazon-expects-weight-loss-frenzy-boost-pharmacy-unit?utm_source=businessinsider&utm_medium=referral&feed_item_type=news (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x126deb9d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 12:20 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 11:06 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 10:35 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 08:04 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 07:57 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024

<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 01:43 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 10:30 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 10:16 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 10:12 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 07:15 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 06:29 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 05:15 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 15, 2024, 04:50 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 14, 2024, 04:02 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 14, 2024, 03:07 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 14, 2024, 02:58 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 14, 2024, 01:44 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 14, 2024, 11:23 AM</span>
<span class="news-post-quotetime warmG

<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 01:14 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 01:10 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 11:06 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 10:54 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 10:49 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 10:39 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 10:28 AM</span>
An error occurred while fetching or parsing the article: HTTPSConnectionPool(host='markets.businessinsider.comhttps', port=443): Max retries exceeded with url: //seekingalpha.com/news/4088615-technologys-profits-are-skewed-to-the-downside-goldman-sachs?utm_source=businessinsider&utm_medium=referral&feed_item_type=news (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1286680d0>: Failed to establish a new connection: [Errno 8] nodename nor servname p

...

<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 08:38 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 08:34 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 08:14 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 08:12 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 07:58 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 07:39 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 07:35 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 07:29 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 10, 2024, 07:17 AM</span>
An error occurred while fetching or parsing the article: HTTPSConnectionPool(host='markets.businessinsider.comhttps', port=443): Max retries exceeded with url: //seekingalpha.com/news/4088558-4-stocks-to-watch-on-wednesday-baba-nvda-and-more?utm_source=businessinsider&utm_medium=referral&feed_item_type=news (Caused by NewConnectionE

<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 03:43 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 02:47 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 02:30 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 02:06 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 01:33 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 11:05 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 10:24 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 09:50 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 04:22 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 1, 2024, 04:00 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 31, 2024, 11:00 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 31, 2024, 10:32 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 31, 2024, 09:00 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 

<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 05:30 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 05:30 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 05:11 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 26, 2024, 05:05 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 25, 2024, 05:36 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 25, 2024, 04:50 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 25, 2024, 04:18 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 25, 2024, 04:16 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 25, 2024, 03:00 PM</span>
AAPL
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 03:22 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 11:45 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 16, 2024, 08:45 AM</span>
An error occurred while fetching or parsing the article: HTTPSConnectionPool(host='markets.businessinsider.

<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 03:19 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 03:14 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 03:01 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 02:53 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 01:21 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 01:00 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 12:27 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 09:09 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 04:45 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 04:30 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 4, 2024, 04:00 AM</span>
<span class="news-post-quotetime warmGrey">Apr. 3, 2024, 10:06 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 3, 2024, 02:55 PM</span>
<span class="news-post-quotetime warmGrey">Apr. 3, 

<span class="news-post-quotetime warmGrey">Mar. 21, 2024, 09:13 AM</span>
An error occurred while fetching or parsing the article: HTTPSConnectionPool(host='markets.businessinsider.comhttps', port=443): Max retries exceeded with url: //seekingalpha.com/news/4082066-4-stocks-to-watch-on-thursday-rddt-market-debut-nke-and-more?utm_source=businessinsider&utm_medium=referral&feed_item_type=news (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x12db52050>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
<span class="news-post-quotetime warmGrey">Mar. 21, 2024, 07:50 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 21, 2024, 05:45 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 21, 2024, 05:40 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 21, 2024, 05:05 AM</span>
<span class="news-post-quotetime warmGrey">Mar. 20, 2024, 05:35 PM</span>
<span class="news-post-quotetime warmGrey">Mar. 2

In [33]:
print(df['articleInfo'])

0     The commercial real estate industry continues ...
1     InvestorPlace - Stock Market News, Stock Advic...
2     InvestorPlace - Stock Market News, Stock Advic...
3     As the first-quarter (Q1) 2024 earnings season...
4     J.P. Morgan analyst Doug Anmuth maintained a B...
5     InvestorPlace - Stock Market News, Stock Advic...
6     Analysts have been eager to weigh in on the Co...
7     Needham analyst Laura Martin initiated coverag...
8     As the tech sector braces for its upcoming ear...
9     InvestorPlace - Stock Market News, Stock Advic...
10    InvestorPlace - Stock Market News, Stock Advic...
11    InvestorPlace - Stock Market News, Stock Advic...
12    InvestorPlace - Stock Market News, Stock Advic...
13    InvestorPlace - Stock Market News, Stock Advic...
14    InvestorPlace - Stock Market News, Stock Advic...
15    InvestorPlace - Stock Market News, Stock Advic...
16    InvestorPlace - Stock Market News, Stock Advic...
17    InvestorPlace - Stock Market News, Stock A

In [53]:
df.to_csv('stockNews.csv')
p_df.to_csv('paragraph.csv')